# Working with WY Aggregated Data

Pre-processing input data for a smoother upload experience of the state data to the WaDE 2.0 database.
Using geopandas to read in shp file, and coverting to WKT for ReportingUnit geometry.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Wyoming/AggregatedAmounts/RawInputData"
os.chdir(workingDir)

# Water Use Data

In [2]:
#CSV input file
fileInput = "WYAggData_input.csv"

In [3]:
# check input 2011
df = pd.read_csv(fileInput)
df

,Water Use by Basin,UseType,Source,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,1334.272368,1498.365835,1226.612777,1.617425e+03,1.846693e+03,2.059991e+03,1826.625609,1417.511879,1.863083e+03,2.106901e+03,1934.922563,2.258502e+03,1526.792191,1546.331897,2.196059e+03,1.746464e+03,2.270430e+03,2.287420e+03,2.110106e+03
1,Bear River Planning Basin,Agricultural Consumptive Use,Surface Water,64817.547130,72789.035020,59587.557520,7.857281e+04,8.971042e+04,1.000722e+05,88735.549340,68861.234970,9.050660e+04,1.023510e+05,93996.501340,1.097157e+05,74169.957510,75119.175850,1.066822e+05,8.484137e+04,1.102951e+05,1.111205e+05,1.025067e+05
2,Bear River Planning Basin,Domestic Use,Ground Water,483.902152,477.808479,481.844024,4.794227e+02,4.795034e+02,4.803509e+02,485.839213,497.179093,5.081558e+02,5.326956e+02,534.169354,5.275973e+02,529.150687,528.035428,5.254066e+02,5.232159e+02,5.207464e+02,5.152099e+02,5.084387e+02
3,Bear River Planning Basin,Domestic Use,Surface Water,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,Bear River Planning Basin,Industrial Use,Ground Water,NaN,90.000000,90.284980,8.765735e+01,8.806528e+01,8.253799e+01,81.340727,77.535625,7.869654e+01,7.373144e+01,78.854391,2.950000e+01,32.347221,29.819371,3.099536e+01,2.622854e+01,2.575507e+01,2.530838e+01,2.477665e+01
5,Bear River Planning Basin,Industrial Use,Surface Water,NaN,310.000000,307.913865,2.471451e+02,1.999903e+02,1.565662e+02,125.147668,76.604210,0.000000e+00,0.000000e+00,0.000000,5.000000e+00,5.482580,5.054131,5.253450e+00,4.445516e+00,4.365266e+00,4.289555e+00,4.199432e+00
6,Bear River Planning Basin,Municipal Use,Ground Water,40.000000,39.458414,39.071567,3.899420e+01,3.930368e+01,3.953578e+01,40.077369,40.773694,4.147002e+01,6.920000e+02,688.723485,6.874129e+02,690.034091,693.965909,7.037955e+02,7.037955e+02,7.077273e+02,7.005189e+02,6.952765e+02
7,Bear River Planning Basin,Municipal Use,Surface Water,1000.000000,987.449887,996.601011,9.914590e+02,9.912846e+02,9.928534e+02,1004.096218,1027.801987,1.050723e+03,2.408000e+03,2416.210424,2.384346e+03,2391.188180,2384.541646,2.368707e+03,2.357956e+03,2.344662e+03,2.319640e+03,2.287971e+03
8,Green River Planning Basin,Agricultural Consumptive Use,Ground Water,2146.650264,2409.417943,1865.810695,2.829314e+03,3.065077e+03,3.260852e+03,2745.512765,2173.316877,2.936011e+03,3.407016e+03,3058.851513,3.507923e+03,2151.805889,2465.718828,3.650256e+03,3.051024e+03,3.602562e+03,4.003081e+03,3.679038e+03
9,Green River Planning Basin,Agricultural Consumptive Use,Surface Water,284204.324700,318993.275800,247022.758100,3.745852e+05,4.057988e+05,4.317183e+05,363490.324700,287734.833100,3.887112e+05,4.510697e+05,404974.598500,4.644291e+05,284886.900400,326447.193700,4.832731e+05,4.039383e+05,4.769587e+05,5.299852e+05,4.870837e+05


In [4]:
droplist = ["2000",
"2001",
"2002",
"2003",
"2004",
"2005",
"2006",
"2007",
"2008",
"2009",
"2010",
"2011",
"2012",
"2013",
"2014",
"2015",
"2016",
"2017",
"2018"]

dftempK = df
dftempK = dftempK.drop(columns=droplist).reset_index()
dftempK

,index,Water Use by Basin,UseType,Source
0,0,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water
1,1,Bear River Planning Basin,Agricultural Consumptive Use,Surface Water
2,2,Bear River Planning Basin,Domestic Use,Ground Water
3,3,Bear River Planning Basin,Domestic Use,Surface Water
4,4,Bear River Planning Basin,Industrial Use,Ground Water
5,5,Bear River Planning Basin,Industrial Use,Surface Water
6,6,Bear River Planning Basin,Municipal Use,Ground Water
7,7,Bear River Planning Basin,Municipal Use,Surface Water
8,8,Green River Planning Basin,Agricultural Consumptive Use,Ground Water
9,9,Green River Planning Basin,Agricultural Consumptive Use,Surface Water


In [5]:
dftempY = df
dftempY = dftempY.drop(columns=['Water Use by Basin', 'UseType', 'Source'])
dftempY

# dftempY = dftempY.T
# dftempY

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,1334.272368,1498.365835,1226.612777,1.617425e+03,1.846693e+03,2.059991e+03,1826.625609,1417.511879,1.863083e+03,2.106901e+03,1934.922563,2.258502e+03,1526.792191,1546.331897,2.196059e+03,1.746464e+03,2.270430e+03,2.287420e+03,2.110106e+03
1,64817.547130,72789.035020,59587.557520,7.857281e+04,8.971042e+04,1.000722e+05,88735.549340,68861.234970,9.050660e+04,1.023510e+05,93996.501340,1.097157e+05,74169.957510,75119.175850,1.066822e+05,8.484137e+04,1.102951e+05,1.111205e+05,1.025067e+05
2,483.902152,477.808479,481.844024,4.794227e+02,4.795034e+02,4.803509e+02,485.839213,497.179093,5.081558e+02,5.326956e+02,534.169354,5.275973e+02,529.150687,528.035428,5.254066e+02,5.232159e+02,5.207464e+02,5.152099e+02,5.084387e+02
3,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,NaN,90.000000,90.284980,8.765735e+01,8.806528e+01,8.253799e+01,81.340727,77.535625,7.869654e+01,7.373144e+01,78.854391,2.950000e+01,32.347221,29.819371,3.099536e+01,2.622854e+01,2.575507e+01,2.530838e+01,2.477665e+01
5,NaN,310.000000,307.913865,2.471451e+02,1.999903e+02,1.565662e+02,125.147668,76.604210,0.000000e+00,0.000000e+00,0.000000,5.000000e+00,5.482580,5.054131,5.253450e+00,4.445516e+00,4.365266e+00,4.289555e+00,4.199432e+00
6,40.000000,39.458414,39.071567,3.899420e+01,3.930368e+01,3.953578e+01,40.077369,40.773694,4.147002e+01,6.920000e+02,688.723485,6.874129e+02,690.034091,693.965909,7.037955e+02,7.037955e+02,7.077273e+02,7.005189e+02,6.952765e+02
7,1000.000000,987.449887,996.601011,9.914590e+02,9.912846e+02,9.928534e+02,1004.096218,1027.801987,1.050723e+03,2.408000e+03,2416.210424,2.384346e+03,2391.188180,2384.541646,2.368707e+03,2.357956e+03,2.344662e+03,2.319640e+03,2.287971e+03
8,2146.650264,2409.417943,1865.810695,2.829314e+03,3.065077e+03,3.260852e+03,2745.512765,2173.316877,2.936011e+03,3.407016e+03,3058.851513,3.507923e+03,2151.805889,2465.718828,3.650256e+03,3.051024e+03,3.602562e+03,4.003081e+03,3.679038e+03
9,284204.324700,318993.275800,247022.758100,3.745852e+05,4.057988e+05,4.317183e+05,363490.324700,287734.833100,3.887112e+05,4.510697e+05,404974.598500,4.644291e+05,284886.900400,326447.193700,4.832731e+05,4.039383e+05,4.769587e+05,5.299852e+05,4.870837e+05


In [6]:
dftempY = dftempY.T.unstack().reset_index(level=1, name='Amount').rename(columns={'level_1':'ReportYear'})[['Amount','ReportYear']]
dftempY

,Amount,ReportYear
0,1334.272368,2000
0,1498.365835,2001
0,1226.612777,2002
0,1617.425105,2003
0,1846.693415,2004
...,...,...
56,8000.927593,2014
56,7961.910748,2015
56,7999.024333,2016
56,7972.759334,2017


In [7]:
dftempY['Water Use by Basin'] = dftempK['Water Use by Basin']
dftempY['UseType'] = dftempK['UseType']
dftempY['Source'] = dftempK['Source']

dftempY

,Amount,ReportYear,Water Use by Basin,UseType,Source
0,1334.272368,2000,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water
0,1498.365835,2001,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water
0,1226.612777,2002,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water
0,1617.425105,2003,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water
0,1846.693415,2004,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water
...,...,...,...,...,...
56,8000.927593,2014,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water
56,7961.910748,2015,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water
56,7999.024333,2016,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water
56,7972.759334,2017,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water


## WaDE Custom Elements (due to missing sate info)

In [8]:
# Creating WaDE Custom reporting u nit native ID for easy identificaiion
# ----------------------------------------------------------------------------------------------------

# Create temp ReportingUnit native id dataframe of unique site.
def assignReportNID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEWY_RU" + string1
    return outstring

dfReportNativeID = pd.DataFrame()

dfReportNativeID['in_ReportingUnitName'] = dftempY['Water Use by Basin']
dfReportNativeID = dfReportNativeID.drop_duplicates()

dftemp = pd.DataFrame(index = dfReportNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfReportNativeID['in_ReportingUnitNativeID'] = dftemp.apply(lambda row: assignReportNID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom ReportingUnit native id
def retrieveReportNID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfReportNativeID.loc[(dfReportNativeID['in_ReportingUnitName'] == A), 'in_ReportingUnitNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dftempY['in_ReportingUnitNativeID'] = dftempY.apply(lambda row: retrieveReportNID(row['Water Use by Basin']), axis=1)
dftempY

,Amount,ReportYear,Water Use by Basin,UseType,Source,in_ReportingUnitNativeID
0,1334.272368,2000,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1
0,1498.365835,2001,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1
0,1226.612777,2002,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1
0,1617.425105,2003,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1
0,1846.693415,2004,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1
...,...,...,...,...,...,...
56,8000.927593,2014,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water,WaDEWY_RU7
56,7961.910748,2015,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water,WaDEWY_RU7
56,7999.024333,2016,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water,WaDEWY_RU7
56,7972.759334,2017,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water,WaDEWY_RU7


In [9]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEWY_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = dftempY['Source']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceTypeCV'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dftempY['in_WaterSourceNativeID'] = dftempY.apply(lambda row: retrieveWaterSourceNativeID(row['Source']), axis=1)
dftempY

,Amount,ReportYear,Water Use by Basin,UseType,Source,in_ReportingUnitNativeID,in_WaterSourceNativeID
0,1334.272368,2000,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1,WaDEWY_WS1
0,1498.365835,2001,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1,WaDEWY_WS1
0,1226.612777,2002,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1,WaDEWY_WS1
0,1617.425105,2003,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1,WaDEWY_WS1
0,1846.693415,2004,Bear River Planning Basin,Agricultural Consumptive Use,Ground Water,WaDEWY_RU1,WaDEWY_WS1
...,...,...,...,...,...,...,...
56,8000.927593,2014,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water,WaDEWY_RU7,WaDEWY_WS2
56,7961.910748,2015,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water,WaDEWY_RU7,WaDEWY_WS2
56,7999.024333,2016,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water,WaDEWY_RU7,WaDEWY_WS2
56,7972.759334,2017,Wind-Bighorn River Planning Basin,Municipal Use,Surface Water,WaDEWY_RU7,WaDEWY_WS2


# Shapefile Data

In [10]:
# Shapefile input
shpInput = "WYBasinShapefile/BasinShapefile.shp"
BasinShape = gpd.read_file(shpInput)
BasinShape

,ReportingU,Reportin_1,Reportin_2,Reportin_3,Reportin_4,StateCV,EPSGCodeCV,Shape_Leng,Shape_Area,TypeIDNum,TypeNameNu,OBJECTID,Reportin_5,Reportin_6,Reportin_7,Reportin_8,Reportin_9,StateCV_1,EPSGCode_1,TypeName_1,TypeIDNum_,geometry
0,20222,WY_1,1,Bear River Planning Basin,Basin,WY,EPSG:4326,4.782684,0.421512,3_1_47,3_Bear River Planning Basin_47,1213,21511,WY_1,1,Bear River Planning Basin,Basin,WY,EPSG:4326,3_Bear River Planning Basin_47,3_1_47,"POLYGON ((-110.74937 42.60565, -110.72887 42.5..."
1,20223,WY_2,2,Green River Planning Basin,Basin,WY,EPSG:4326,15.277180,5.913140,3_2_47,3_Green River Planning Basin_47,1214,21512,WY_2,2,Green River Planning Basin,Basin,WY,EPSG:4326,3_Green River Planning Basin_47,3_2_47,"POLYGON ((-110.02174 43.43713, -109.99858 43.4..."
2,20224,WY_3,3,Northeast River Planning Basin,Basin,WY,EPSG:4326,9.138325,3.457522,3_3_47,3_Northeast River Planning Basin_47,1215,21513,WY_3,3,Northeast River Planning Basin,Basin,WY,EPSG:4326,3_Northeast River Planning Basin_47,3_3_47,"POLYGON ((-105.00107 45.00031, -104.05819 44.9..."
3,20225,WY_4,4,Platte River Planning Basin,Basin,WY,EPSG:4326,15.864408,6.772901,3_4_47,3_Platte River Planning Basin_47,1216,21514,WY_4,4,Platte River Planning Basin,Basin,WY,EPSG:4326,3_Platte River Planning Basin_47,3_4_47,"POLYGON ((-106.52719 43.19357, -106.52964 43.1..."
4,20226,WY_5,5,Powder-Tongue River Planning Basin,Basin,WY,EPSG:4326,11.088498,3.198029,3_5_47,3_Powder-Tongue River Planning Basin_47,1217,21515,WY_5,5,Powder-Tongue River Planning Basin,Basin,WY,EPSG:4326,3_Powder-Tongue River Planning Basin_47,3_5_47,"POLYGON ((-105.85433 45.00043, -105.04859 44.9..."
5,20227,WY_6,6,Salt-Snake River Planning Basin,Basin,WY,EPSG:4326,7.920928,1.481469,3_6_47,3_Salt-Snake River Planning Basin_47,1218,21516,WY_6,6,Salt-Snake River Planning Basin,Basin,WY,EPSG:4326,3_Salt-Snake River Planning Basin_47,3_6_47,"POLYGON ((-110.63437 44.48433, -110.61510 44.4..."
6,20228,WY_7,7,Wind-Bighorn River Planning Basin,Basin,WY,EPSG:4326,14.956223,6.730750,3_7_47,3_Wind-Bighorn River Planning Basin_47,1219,21517,WY_7,7,Wind-Bighorn River Planning Basin,Basin,WY,EPSG:4326,3_Wind-Bighorn River Planning Basin_47,3_7_47,"POLYGON ((-109.10415 45.00585, -109.06805 44.9..."


In [11]:
#Create geo output
columnsList = ['RU_Name', 'geometry']
dfshape = pd.DataFrame(columns=columnsList)
dfshape['RU_Name'] = BasinShape['Reportin_3']
dfshape['geometry'] = BasinShape['geometry']
dfshape = dfshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfshape

,RU_Name,geometry
0,Bear River Planning Basin,"POLYGON ((-110.74937 42.60565, -110.72887 42.5..."
1,Green River Planning Basin,"POLYGON ((-110.02174 43.43713, -109.99858 43.4..."
2,Northeast River Planning Basin,"POLYGON ((-105.00107 45.00031, -104.05819 44.9..."
3,Platte River Planning Basin,"POLYGON ((-106.52719 43.19357, -106.52964 43.1..."
4,Powder-Tongue River Planning Basin,"POLYGON ((-105.85433 45.00043, -105.04859 44.9..."
5,Salt-Snake River Planning Basin,"POLYGON ((-110.63437 44.48433, -110.61510 44.4..."
6,Wind-Bighorn River Planning Basin,"POLYGON ((-109.10415 45.00585, -109.06805 44.9..."


### Inspect Output Data & Export

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dftempY.dtypes)

Amount                      float64
ReportYear                   object
Water Use by Basin           object
UseType                      object
Source                       object
in_ReportingUnitNativeID     object
in_WaterSourceNativeID       object
dtype: object


In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfshape.dtypes)

RU_Name       object
geometry    geometry
dtype: object


In [14]:
# Export out to CSV.
dftempY.to_csv('P_wyAggMaster.csv', index=False) # The output.
dfshape.to_csv('P_wyGeometry.csv', index=False) # The output geometry.